In [ ]:
!apt-get update
!apt install -y xvfb
# Install necessary libraries
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install pyvirtualdisplay
!pip install google-cloud-vision google-cloud-storage google-cloud-translate google-cloud-automl

In [17]:

import os
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image


import xlsxwriter


import vertexai
from vertexai.language_models import TextGenerationModel
import pandas as pd
from image_analysis import analyze_image_for_criteria


import streamlit as st
import pandas as pd
import os
import io

from selenium import webdriver
#from PIL import Image

from google.cloud import vision
from selenium import webdriver


import vertexai
from vertexai.language_models import TextGenerationModel

from google.api_core.client_options import ClientOptions

In [ ]:
!pip install google-cloud-aiplatform


In [ ]:
!pip install google-cloud-storage google-cloud-aiplatform


## Save image in png + display

In [12]:
from selenium import webdriver
import os

# Define Chrome options for headless mode
chrome_options = webdriver.ChromeOptions()
from selenium.webdriver.firefox.options import Options

#options = Options()
chrome_options.add_argument('--disable-blink-features=AutomationControlled')

chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize the Chrome driver with the defined options
driver = webdriver.Chrome(options=chrome_options)

# Function to try to close cookie notice
def close_cookie_notice(driver):
    try:
        accept_button = driver.find_element_by_xpath("//button[contains(text(), 'zustimmen')]")
        accept_button.click()
    except Exception as e:
        print("Could not find or click cookie acceptance button:", e)

# Function to capture, display and save full-page screenshot
def capture_and_display_fullpage_screenshot(url):
    driver.get(url)

    # Try to close cookie notice if it exists
    close_cookie_notice(driver)

    # Trigger JavaScript to get the full page screenshot
    result = driver.execute_script("return document.body.parentNode.scrollHeight")
    driver.set_window_size(800, result)  # Width, Height
    png = driver.get_screenshot_as_png()

    # Save the screenshot to a file
    screenshot_path = "screenshotZ2.png"  # Local path
    #os.makedirs(os.path.dirname(screenshot_path), exist_ok=True)
    with open(screenshot_path, "wb") as file:
        file.write(png)

    print(f"Screenshot saved at {screenshot_path}")
    

    # Cleanup
    driver.quit()

    return png

# Replace with your desired URL
url = "https://www.houseofshoes.de/"
screenshot_data=capture_and_display_fullpage_screenshot(url)

Could not find or click cookie acceptance button: 'WebDriver' object has no attribute 'find_element_by_xpath'
Screenshot saved at screenshotZ2.png


## save them in variables

# On multiple URLs (later)

#VERTEX AI

In [5]:
import os

# Assuming the filename is 'your-service-account-key.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "GCP_key.json"


#testing the api

In [ ]:
from google.cloud import vision

def analyze_image(image_path):
    """Analyzes an image using Google Vision API."""

    client = vision.ImageAnnotatorClient()

    # Load the image
    with open(image_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    # Perform label detection on the image file
    response = client.label_detection(image=image)

    labels = response.label_annotations
    print('Labels:')
    for label in labels:
        print(label.description)

# Example usage
analyze_image('screenshotZ.png')


# On stored image

In [ ]:
from google.cloud import vision

def analyze_image(image_data):
    """Analyzes an image using Google Vision API."""

    client = vision.ImageAnnotatorClient()

    # Use the byte array of the image directly
    image = vision.Image(content=image_data)

    # Perform label detection on the image
    response = client.label_detection(image=image)

    labels = response.label_annotations
    print('Labels:')
    for label in labels:
        print(label.description)

# Assuming screenshot_data contains the byte array of the screenshot
# Replace screenshot_data with the variable containing your image data
analyze_image(screenshot_data)


# On stored image

## store in a df

In [15]:

def analyze_image_for_textual_criterias(image_data):
    """Analyze the image for specific text criteria and store results in a DataFrame."""
    client = vision.ImageAnnotatorClient()

    # Use the byte array of the image directly
    image = vision.Image(content=image_data)
    response = client.text_detection(image=image)
    texts = response.text_annotations

    if not texts:
        print("No text detected in the image.")
        return

    # Extracting the first element which contains the entire extracted text
    full_text = texts[0].description.lower()

    # Define your criteria
    criteria = {
        "trusted shops": "trusted shops" in full_text,
        "sale or similar": any(word in full_text for word in ["sale", "rabatt", "ermäßigung", "schlussverkauf"]),
        "return or similar": any(word in full_text for word in ["return", "rücksendung", "rückversand", "rückgabe", "rücksendung informationen", "rückerstattung"]),
        "free returns": "free returns" in full_text or "rücksendung kostenlos" in full_text or "kostenlose lieferung und rücksendung" in full_text,
        "free delivery": any(word in full_text for word in ["kostenlose lieferung", "gratisversand", "standardlieferung - kostenlos ab", "kostenfreier versand", "kostenloser versand ab", "kostenloser versand"]),
        "FAQ": "faq" in full_text or "fragen und antworten" in full_text or "fragen & antworten" in full_text,
    }

    # Convert the dictionary to a DataFrame
    criteria_df = pd.DataFrame(list(criteria.items()), columns=['Criteria', 'Result (1=Yes, 0=No)'])

    return full_text,criteria_df


In [ ]:
from google.cloud import vision
import pandas as pd
# Replace screenshot_data with the variable containing your image data
full_text, df2 = analyze_image_for_textual_criterias(screenshot_data)

# To view the DataFrame
print(df2)


# Logos detection

In [ ]:
def detect_logos(image_data):
    """Detect logos in an image using Google Cloud Vision API."""
    client = vision.ImageAnnotatorClient()
    image = vision.Image(content=image_data)

    # Perform logo detection
    response = client.logo_detection(image=image)
    logos = response.logo_annotations

    if not logos:
        print("No logos detected.")
        return []

    detected_logos = [logo.description.lower() for logo in logos]
    return detected_logos

# Later in your code, you can call this function
logos = detect_logos(screenshot_data)

# Check for specific logos
contains_visa = "visa" in logos
contains_mastercard = "mastercard" in logos
contains_paypal = "paypal" in logos


## from image path 

In [ ]:
from google.cloud import vision

def detect_logos_from_path(image_path):
    """Detect logos in an image file using Google Cloud Vision API."""
    client = vision.ImageAnnotatorClient()

    # Load the image from the file path
    with open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    # Perform logo detection
    response = client.logo_detection(image=image)
    logos = response.logo_annotations

    if response.error.message:
        raise Exception(f'Error in logo detection: {response.error.message}')

    detected_logos = [logo.description for logo in logos]
    return detected_logos

# Example usage
image_path = 'logos_test.png'
logos = detect_logos_from_path(image_path)
print("Detected logos:", logos)


# red colored criterias : 

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel


def interview(
    temperature: float,
    project_id: str,
    location: str,
) -> str:
    """Ideation example with a Large Language Model"""

    vertexai.init(project=project_id, location=location)
    # TODO developer - override these parameters as needed:
    parameters = {
        "temperature": temperature,  # Temperature controls the degree of randomness in token selection.
        "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
        "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
        "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        "Give me ten interview questions for the role of program manager.",
        **parameters,
    )
    print(f"Response from Model: {response.text}")

    return response.text


if __name__ == "__main__":
    interview(0.5,'jovial-circuit-412017','us-central1')

In [ ]:
full_text

In [19]:
import vertexai
from vertexai.language_models import TextGenerationModel
import pandas as pd

def interview_and_store(
    temperature: float,
    project_id: str,
    location: str,
    full_text: str
) -> pd.DataFrame:

    vertexai.init(project=project_id, location=location)
    parameters = {
        "temperature": temperature,
        "max_output_tokens": 256,
        "top_p": 0.8,
        "top_k": 40,
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")

    # Create prompts for each of the criteria, including "trusted shops"
     # Create prompts for each of the criteria, including "trusted shops"
    prompts = [
        f"In English, you're a German language expert. Is the following text in German? Respond with yes or no and say why in 1 sentence: {full_text}",
        f"In English, does the following text contain information related to delivery? Respond with yes or no and give the delivery information: {full_text}",
        f"In English, does the following text contain a phone number? Answer with yes or no and if yes, provide the phone number: {full_text}",
    
        f"{full_text} In English and be specific, is there any text that contains 'sale', 'Rabatt', or 'Ermäßigung' or 'Schlussverkauf' or similar (in any English or German language) in this text, and where is it located in the text (give me the German original text)? Respond with yes or no. If yes, provide its location. This text is extracted using OCR on an image. Where is its location in the image?",
    
        f"{full_text} In English and be specific, is there any text that contains 'return', 'Rücksendung', 'Rückversand', 'Rückgabe', 'Rücksendung Informationen', 'Rückerstattung' or similar (in any English or German language) in this text, and where is it located in the text (give me the German original text)? Respond with yes or no. If yes, provide its location. This text is extracted using OCR on an image. Where is its location in the image?",
    
        f"{full_text} In English and be specific, is there any text that contains 'free returns', 'Rücksendung kostenlos', 'Kostenlose Lieferung und Rücksendung' or similar (in any English or German language) in this text, and where is it located in the text (give me the German original text)? Respond with yes or no. If yes, provide its location. This text is extracted using OCR on an image. Where is its location in the image?",
    
        f"{full_text} In English and be specific, is there any text that contains 'Kostenlose Lieferung', 'gratisversand', 'Kostenlose Lieferung', 'gratisversand', 'Standardlieferung - Kostenlos ab', 'Kostenfreier Versand', 'Kostenloser Versand ab', 'Kostenloser Versand' or similar (in any English or German language) in this text, and where is it located in the text (give me the German original text)? Respond with yes or no. If yes, provide its location. This text is extracted using OCR on an image. Where is its location in the image?",
    
        f"{full_text} In English and be specific, is there any text that contains 'trusted shops' (in any English or German language) in this text, and where is it located in the text (give me the German original text)? Respond with yes or no. If yes, provide its location. This text is extracted using OCR on an image. Where is its location in the image?"
    ]

    results = []

    for prompt in prompts:
        response = model.predict(prompt, **parameters)
        response_text = response.text.lower()
        print(response_text)
        # Check if the response contains "yes" or "no" and extract additional information if needed
        if "yes" in response_text:
            criteria_result = "yes"
            additional_info = response_text.split("yes")[1].strip()
        elif "no" in response_text:
            criteria_result = "no"
            additional_info = "No"
        else:
            criteria_result = "unknown"
            additional_info = "Unknown"

        results.append({
            "criteria": prompt.split('?')[0],  # Extract the criteria from the prompt
            "yes or no": criteria_result,
            "additional infos": additional_info
        })

    # Create a DataFrame from the results
    df = pd.DataFrame(results)

    return df

# Gemini Vision

In [ ]:
from google.cloud import aiplatform
from google.oauth2 import service_account

# Path to your service account key file
key_path = "GCP_key.json"

# Authenticate the client
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# Initialize the client
client = aiplatform.gapic.PredictionServiceClient(credentials=credentials)

# Your GCP project and region
project = "jovial-circuit-412017"
region = "us-central1"

# Construct the endpoint resource name
endpoint = client.endpoint_path(
    project=project,
    location=region,
    endpoint="ENDPOINT_ID"  # Replace with your endpoint ID
)

# The path to the image file in Google Cloud Storage
image_gcs_path = "gs://YOUR_BUCKET_NAME/YOUR_IMAGE_PATH"

# The questions you want to ask
questions = [
    "Does the image contain a chat support icon?",
    "Does this image contain images from payment brands Visa, Mastercard, PayPal?",
    "Does this image contain images from payment brands Klarna, Sofort, Giropay?"
]

# Make the prediction request
response = client.predict(
    endpoint=endpoint,
    instances=[{
        "content": image_gcs_path,
        "mimeType": "image/jpeg",  # or the appropriate type of your image
        "questions": questions
    }],
)

# Parse and print the response
for prediction in response.predictions:
    print("Prediction results:", prediction)


# Other

In [ ]:
from google.cloud import storage

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

# Replace these variables with your information
bucket_name = "mounim_bucket"
source_file_name = "screenshotZ.png"
destination_blob_name = "image_blob"

upload_blob(bucket_name, source_file_name, destination_blob_name)


In [ ]:
from google.cloud import aiplatform

def analyze_image(project, location, endpoint_id, gcs_uri):
    client = aiplatform.gapic.PredictionServiceClient()

    endpoint = client.endpoint_path(
        project=project,
        location=location,
        endpoint=endpoint_id,
    )

    instances = [{
        "content": gcs_uri,
        "mimeType": "image/jpeg",
        "questions": [
            "Does the image contain a chat support icon?",
            "Does this image contain images from payment brands Visa, Mastercard, PayPal?",
            "Does this image contain images from payment brands Klarna, Sofort, Giropay?"
        ]
    }]

    response = client.predict(endpoint=endpoint, instances=instances)
    return response


project = "jovial-circuit-412017"
region = "us-central1"

endpoint_id = "your-endpoint-id"
gcs_uri = f"gs://{bucket_name}/{destination_blob_name}"

result = analyze_image(project, location, endpoint_id, gcs_uri)
print(result)


# Directly from vertex AI doc

In [ ]:
!pip3 install --upgrade --user google-cloud-aiplatform

In [ ]:
!pip install --upgrade google-cloud-aiplatform

In [ ]:
from google.cloud import aiplatform

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "GCP_key.json"


In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image

PROJECT_ID = "jovial-circuit-412017"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

IMAGE_FILE = "screenshotZ.png"
image = Image.load_from_file(IMAGE_FILE)

generative_multimodal_model = GenerativeModel("gemini-pro-vision")
response = generative_multimodal_model.generate_content(["What is shown in this image?", image])

print(response)

In [ ]:
response1 = generative_multimodal_model.generate_content(["Scan for delivery logos: DHL, Hermes, DPD, UPS, FedEx, Deutsche Post. Yes or no? If yes, which and where?", image])

In [ ]:
response2 = generative_multimodal_model.generate_content(["Detect payment logos: Visa, Mastercard, PayPal. Present, yes or no? If yes, specify brands and locations.", image])

In [ ]:
response3 = generative_multimodal_model.generate_content(["Look for payment logos: Klarna, Sofort, Giropay. Are they in the image? If yes, identify brands and their locations.", image])

In [ ]:
response1.text

In [ ]:
response2.text

In [ ]:
response3.text

# improved 

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image

PROJECT_ID = "jovial-circuit-412017"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

IMAGE_FILE = "screenshotZ.png"
image = Image.load_from_file(IMAGE_FILE)


# Initialize Vertex AI
PROJECT_ID = "jovial-circuit-412017"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

# Load the image
IMAGE_FILE = "screenshotZ.png"
image = Image.load_from_file(IMAGE_FILE)

# Initialize the Gemini Pro Vision model
generative_multimodal_model = GenerativeModel("gemini-pro-vision")

# Define your custom prompts
prompts = [
    "Scan for delivery logos: DHL, Hermes, DPD, UPS, FedEx, Deutsche Post. Yes or no? If yes, which and where?",
    "Detect payment logos: Visa, Mastercard, PayPal. Present, yes or no? If yes, specify brands and locations.",
    "Look for payment logos: Klarna, Sofort, Giropay. Are they in the image? If yes, identify brands and their locations."
]

# Analyze the image using each prompt
for prompt in prompts:
    response = generative_multimodal_model.generate_content([prompt, image])
    print(f"Prompt: {prompt}\nResponse: {response.text}")

# Note: Depending on the model's capabilities, you might need to further process the response to extract the exact details you need.


In [ ]:
for prompt in prompts:
    response = generative_multimodal_model.generate_content([prompt, image])
    response_text = response.text  # Assuming 'response.text' contains the text response from the model

    # Check if the response contains "yes" or "no"
    yes_no = "yes" if "yes" in response_text.lower() else "no" if "no" in response_text.lower() else "unknown"

    # Append the result to the data list
    data.append({"criteria": prompt, "yes/no": yes_no, "additional_infos": response_text})

# Create a DataFrame
df3 = pd.DataFrame(data)

# Print the DataFrame
print(df3)

# ALLL

In [20]:
#url = "https://www.houseofshoes.de/"
#screenshot_data = capture_and_return_fullpage_screenshot(url)
full_text, df2 = analyze_image_for_textual_criterias(screenshot_data)
df = interview_and_store(0.1, 'jovial-circuit-412017', 'us-central1', full_text)


  yes, the text is in german.

the text is about a website that sells comfortable shoes. it has a lot of information about different brands of shoes, as well as a variety of different styles. the website also has a newsletter that you can sign up for to receive updates about new products and sales.

  yes, the text contains information related to delivery.

the text states that the delivery costs are 6.95 eur and that returns are free.

 0201 - 269 700 30
yes, the phone number 0201 - 269 700 30 is mentioned in the text.
yes, there is a text that contains 'sale' in the german language. it is located in the following sentence:
"sale - 20% auf alle schuhe"
yes, there is a text that contains "rücksendung" in the german language. it is located in the footer of the page.
yes, there is a text that contains 'free returns' in the german language. it is located in the footer of the page, and it says "kostenlose lieferung und rücksendung".
yes, there is a text that contains "kostenlose lieferung

In [21]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image

PROJECT_ID = "jovial-circuit-412017"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

IMAGE_FILE = "screenshotZ.png"
image = Image.load_from_file(IMAGE_FILE)


# Initialize Vertex AI
PROJECT_ID = "jovial-circuit-412017"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

# Load the image
IMAGE_FILE = "screenshotZ.png"
image = Image.load_from_file(IMAGE_FILE)

# Initialize the Gemini Pro Vision model
generative_multimodal_model = GenerativeModel("gemini-pro-vision")

# Define your custom prompts
prompts = [
    "Scan for delivery logos: DHL, Hermes, DPD, UPS, FedEx, Deutsche Post. Yes or no? If yes, which and where?",
    "Detect payment logos: Visa, Mastercard, PayPal. Present, yes or no? If yes, specify brands and locations.",
    "Look for payment logos: Klarna, Sofort, Giropay. Are they in the image? If yes, identify brands and their locations."
]

# Analyze the image using each prompt
for prompt in prompts:
    response = generative_multimodal_model.generate_content([prompt, image])
    print(f"Prompt: {prompt}\nResponse: {response.text}")

# Note: Depending on the model's capabilities, you might need to further process the response to extract the exact details you need.


Prompt: Scan for delivery logos: DHL, Hermes, DPD, UPS, FedEx, Deutsche Post. Yes or no? If yes, which and where?
Response:  Yes, Hermes, DPD, UPS, Deutsche Post.
In the footer of the website.
Prompt: Detect payment logos: Visa, Mastercard, PayPal. Present, yes or no? If yes, specify brands and locations.
Response:  Yes
Bottom right corner
Prompt: Look for payment logos: Klarna, Sofort, Giropay. Are they in the image? If yes, identify brands and their locations.
Response:  Yes.

1. Klarna - Top right corner of the image.
2. Sofort - Top right corner of the image.
3. Giropay - Top right corner of the image.


In [1]:
def create_endpoint_sample(
    project: str,
    display_name: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
    timeout: int = 300,
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.EndpointServiceClient(client_options=client_options)
    endpoint = {"display_name": display_name}
    parent = f"projects/{project}/locations/{location}"
    response = client.create_endpoint(parent=parent, endpoint=endpoint)
    print("Long running operation:", response.operation.name)
    create_endpoint_response = response.result(timeout=timeout)
    print("create_endpoint_response:", create_endpoint_response)


In [6]:
from google.cloud import aiplatform
    
create_endpoint_sample(project='jovial-circuit-412017',display_name='mounim')

Long running operation: projects/461884919444/locations/us-central1/endpoints/1499533749170208768/operations/6346062188431540224
create_endpoint_response: name: "projects/461884919444/locations/us-central1/endpoints/1499533749170208768"



In [25]:
data=[]

In [26]:
for prompt in prompts:
    response = generative_multimodal_model.generate_content([prompt, image])
    response_text = response.text  # Assuming 'response.text' contains the text response from the model

    # Check if the response contains "yes" or "no"
    yes_no = "yes" if "yes" in response_text.lower() else "no" if "no" in response_text.lower() else "unknown"
    
    # Append the result to the data list
    data.append({"criteria": prompt, "yes/no": yes_no, "additional_infos": response_text})


In [27]:
# Create a DataFrame
df3 = pd.DataFrame(data)

# Print the DataFrame
print(df3)

                                            criteria yes/no  \
0  Scan for delivery logos: DHL, Hermes, DPD, UPS...    yes   
1  Detect payment logos: Visa, Mastercard, PayPal...    yes   
2  Look for payment logos: Klarna, Sofort, Giropa...    yes   

                                    additional_infos  
0   Yes, Hermes, DPD, UPS, Deutsche Post.\nIn the...  
1                           Yes\nBottom right corner  
2   Yes.\n\n1. Klarna - top right corner of the i...  


In [29]:
df

,criteria,yes or no,additional infos
0,"In English, you're a German language expert. I...",yes,", the text is in german.\n\nthe text is about ..."
1,"In English, does the following text contain in...",yes,", the text contains information related to del..."
2,"In English, does the following text contain a ...",yes,", the phone number 0201 - 269 700 30 is mentio..."
3,beratungs-hotline 0201 - 269 700\n30 (montag-f...,yes,", there is a text that contains 'sale' in the ..."
4,beratungs-hotline 0201 - 269 700\n30 (montag-f...,yes,", there is a text that contains ""rücksendung"" ..."
5,beratungs-hotline 0201 - 269 700\n30 (montag-f...,yes,", there is a text that contains 'free returns'..."
6,beratungs-hotline 0201 - 269 700\n30 (montag-f...,yes,", there is a text that contains ""kostenlose li..."
7,beratungs-hotline 0201 - 269 700\n30 (montag-f...,yes,", there is a text that contains 'trusted shops..."


# Finish all

## Csv format

In [30]:
df1=df

In [31]:

#df2.rename(columns={df2.columns[1]: 'yes/no(1/0)'}, inplace=True)
#df2['yes/no(1/0)'] = df2['yes/no(1/0)'].apply(lambda x: 1 if str(x).strip().lower() in ['yes', 'true'] else 0)
# Rename the second column
df1.rename(columns={'yes or no': 'yes/no(1/0)'}, inplace=True)

# Convert values in the 'yes/no(1/0)' column to 1 or 0 based on the criteria
df1['yes/no(1/0)'] = df1['yes/no(1/0)'].apply(lambda x: 1 if str(x).strip().lower() in ['yes', 'true'] else 0)

df1.head()

# Rename the second column and update its values
df3.rename(columns={df3.columns[1]: 'yes/no(1/0)'}, inplace=True)
df3['yes/no(1/0)'] = df3['yes/no(1/0)'].apply(lambda x: 1 if str(x).strip().lower() in ['yes', 'true'] else 0)

df1 = df1.rename(columns={'additional infos': 'additional_infos'})
#df2 = df2.rename(columns={'Criteria': 'criteria'})

# Add an empty column 'additional_infos' to df2
#df2['additional_infos'] = ''


# Display the first few rows of the updated dataframe


In [33]:
import pandas as pd

# Assuming df1 and df2 are your DataFrames

# Remove duplicates based on the "criteria" column from df1
df1 = df1.drop_duplicates(subset="criteria", keep="first")

# Remove duplicates based on the "criteria" column from df2
#df2 = df2.drop_duplicates(subset="criteria", keep="first")

# Concatenate the two DataFrames without duplicates
df3 = df3.drop_duplicates(subset="criteria", keep="first")


In [34]:
result = pd.concat([df1, df2,df3], axis=0, ignore_index=True)


In [41]:
df1.head(20)

,criteria,yes/no(1/0),additional_infos
0,"In English, you're a German language expert. I...",1,", the text is in german.\n\nthe text is about ..."
1,"In English, does the following text contain in...",1,", the text contains information related to del..."
2,"In English, does the following text contain a ...",1,", the phone number 0201 - 269 700 30 is mentio..."
3,beratungs-hotline 0201 - 269 700\n30 (montag-f...,1,", there is a text that contains 'sale' in the ..."
4,beratungs-hotline 0201 - 269 700\n30 (montag-f...,1,", there is a text that contains ""rücksendung"" ..."
5,beratungs-hotline 0201 - 269 700\n30 (montag-f...,1,", there is a text that contains 'free returns'..."
6,beratungs-hotline 0201 - 269 700\n30 (montag-f...,1,", there is a text that contains ""kostenlose li..."
7,beratungs-hotline 0201 - 269 700\n30 (montag-f...,1,", there is a text that contains 'trusted shops..."


In [42]:
df3.head(20)

,criteria,yes/no(1/0),additional_infos
0,"Scan for delivery logos: DHL, Hermes, DPD, UPS...",1,"Yes, Hermes, DPD, UPS, Deutsche Post.\nIn the..."
1,"Detect payment logos: Visa, Mastercard, PayPal...",1,Yes\nBottom right corner
2,"Look for payment logos: Klarna, Sofort, Giropa...",1,Yes.\n\n1. Klarna - top right corner of the i...


In [35]:
import pandas as pd

# Assuming 'result' is your existing DataFrame
# Add 'Company_Name' column at the beginning
result.insert(0, 'Company_Name', 'houseofshoes')

# Add 'Company_Url' column at the second position
result.insert(1, 'Company_Url', 'https://www.houseofshoes.de/')

# Now 'result' has two new columns at the beginning with the specified values


In [36]:
!pip install openpyxl



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
result.head(20)

,Company_Name,Company_Url,criteria,yes/no(1/0),additional_infos,Criteria,"Result (1=Yes, 0=No)"
0,houseofshoes,https://www.houseofshoes.de/,"In English, you're a German language expert. I...",1.0,", the text is in german.\n\nthe text is about ...",NaN,NaN
1,houseofshoes,https://www.houseofshoes.de/,"In English, does the following text contain in...",1.0,", the text contains information related to del...",NaN,NaN
2,houseofshoes,https://www.houseofshoes.de/,"In English, does the following text contain a ...",1.0,", the phone number 0201 - 269 700 30 is mentio...",NaN,NaN
3,houseofshoes,https://www.houseofshoes.de/,beratungs-hotline 0201 - 269 700\n30 (montag-f...,1.0,", there is a text that contains 'sale' in the ...",NaN,NaN
4,houseofshoes,https://www.houseofshoes.de/,beratungs-hotline 0201 - 269 700\n30 (montag-f...,1.0,", there is a text that contains ""rücksendung"" ...",NaN,NaN
5,houseofshoes,https://www.houseofshoes.de/,beratungs-hotline 0201 - 269 700\n30 (montag-f...,1.0,", there is a text that contains 'free returns'...",NaN,NaN
6,houseofshoes,https://www.houseofshoes.de/,beratungs-hotline 0201 - 269 700\n30 (montag-f...,1.0,", there is a text that contains ""kostenlose li...",NaN,NaN
7,houseofshoes,https://www.houseofshoes.de/,beratungs-hotline 0201 - 269 700\n30 (montag-f...,1.0,", there is a text that contains 'trusted shops...",NaN,NaN
8,houseofshoes,https://www.houseofshoes.de/,NaN,NaN,NaN,trusted shops,True
9,houseofshoes,https://www.houseofshoes.de/,NaN,NaN,NaN,sale or similar,True


In [40]:
# Save to CSV
result.to_csv('result_revised.csv', index=False)  # Set index=False to exclude row indices from the CSV

# Save to Excel
result.to_excel('result_revised.xlsx', index=False)  # Set index=False to exclude row indices from the Excel file
